In [1]:
from __future__ import print_function

import datetime
import numpy as np
import glob

from multiprocessing import Pool

## Boilerplate path hack to give access to full clustered_SNe package
import os, sys
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, parse_run, \
                                         Inputs
    
from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs



/Users/egentry/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/egentry/Dropbox/Data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:38: UserWarning: `session' from database_helpers can only write using 1 process at a time
  UserWarning)


#### Helpful resource:
http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

In [2]:
def parse_into_simulations(data_dir_and_id):
    # wrapper function to allow multiprocessing + map
    data_dir, id = data_dir_and_id
    last_run = parse_run(data_dir, id)
    simulation = Simulation.from_last_run(data_dir, last_run)
    
    inputs = Inputs(os.path.join(data_dir, id + "_inputs.dat"))
    simulation_inputs = Simulation_Inputs.from_Inputs(id, inputs)
    return simulation, simulation_inputs
        

## Check which rows already exist

In [ ]:
for sim in session.query(Simulation):
    print(sim)

In [ ]:
for sim_inputs in session.query(Simulation_Inputs):
    print(sim_inputs)

## Add files to database in batch

In [3]:
def add_entire_dir_to_database(data_dir):
    overview_filenames = glob.glob(os.path.join(data_dir, "*overview.dat"))
    ids = [None]*len(overview_filenames)
    for k, filename in enumerate(overview_filenames):
        ids[k] = os.path.basename(filename).split("_overview.dat")[0]
#     print(ids)

    mappable_inputs = [(data_dir, id) for id in ids]
    
    pool = Pool(4)
    simulation_tuples = pool.map(parse_into_simulations, mappable_inputs)
    for simulation, simulation_inputs in simulation_tuples:
        simulation.add_or_update_to_table()
        simulation_inputs.add_or_update_to_table()
add_entire_dir_to_database("../saved_runs/cluster_parameter_study")

In [4]:
session.commit()

In [5]:
session.close()